# Building a multilingual domain-specific vocabulary

Analysing book reviews across multiple languages can shine a light on how readers express their reading experiences in different languages.

Reviewers writing reviews in different languages use different terminology to talk about writing style, narrative, characers and plot, or about what they like and don't like. But there is possibly a lot of overlap between different languages in how reviewers express themselves. That is, there is a **vocabulary for talking about book aspects and reading experiences with links between different languages**.

Although we can probably come up with a set of relevant words for such a vocabulary in the languages that we are fluent in, finding similar terms in different languages is challenging. Because the **register of words** that people use to talk about reading experience probably consists of relatively common words, we should focus on **words that are shared by many reviewers, and that are typical in reviews**. If we zoom in on less common words, we are likely to find words that are specific to a specific book, author or series.

In this notebook, we extract a domain-specific vocabulary of book reviews for a range of languages, using simple statistics.

In [1]:
%reload_ext autoreload
%autoreload 2


First, we import a set of python libraries that will be used.

In [62]:
# python internal libraries
import glob
import gzip
import json
import os
from ast import literal_eval
from collections import Counter
from collections import defaultdict

# external libraries
import numpy as np
import pandas as pd
import spacy
import stopwordsiso

# scripts in this directory
from analyse import read_from_doc_bins
from language import lang_code_map, spacy_model_map
from language import load_language_nlp_model

# The language code map shows the list of languages for which reviews are available.
lang_code_map

{'Arabic': 'ar',
 'Czech': 'cs',
 'Danish': 'da',
 'German': 'de',
 'Greek': 'el',
 'English': 'en',
 'Spanish': 'es',
 'Persian': 'fa',
 'Finnish': 'fi',
 'French': 'fr',
 'Hindi': 'hi',
 'Hungarian': 'hu',
 'Indonesian': 'id',
 'Italian': 'it',
 'Japanese': 'ja',
 'Korean': 'ko',
 'Dutch': 'nl',
 'Norwegian': 'no',
 'Polish': 'pl',
 'Pashto': 'ps',
 'Portuguese': 'pt',
 'Russian': 'ru',
 'Slovak': 'sk',
 'Slovenian': 'sl',
 'Serbian': 'sr',
 'Swedish': 'sv',
 'Turkish': 'tr',
 'Ukranian': 'uk',
 'Urdu': 'ur',
 'Chinese': 'zh'}

## Language and linguistics parser

In this analysis we focus on language for which linguistic parsers are available. [SpaCy](https://spacy.io) has parser models for a range of languages. For some languages, there is no SpaCy model, but linguistics parsers and other language-specific NLP techniques are available elsewhere (e.g. [Farsi/Persian](https://github.com/Dadmatech/DadmaTools), [Arabic](https://github.com/Curated-Awesome-Lists/awesome-arabic-nlp)).

In [84]:
languages = [
    # Add languages for which you want to do linguistic parsing of reviews
    'Danish', 'Dutch', 'English', 'French', 'German', 'Italian', 
    'Japanese', 'Korean', 'Persian', 'Polish', 'Portuguese', 'Slovenian', 'Spanish', 'Swedish', 'Ukranian'
    # Skipping Chinese because the parser seems to reduce everything to a single character
    # 'Chinese'
]

lang_codes = sorted([lang_code_map[language] for language in languages])
lang_codes

['da',
 'de',
 'en',
 'es',
 'fa',
 'fr',
 'it',
 'ja',
 'ko',
 'nl',
 'pl',
 'pt',
 'sl',
 'sv',
 'uk']

## Load the SpaCy models

If you want to parse the reviews yourself, you need to install the various SpaCy models. If you want to use the pre-parsed reviews (in the `spacy_doc_bins` directory), you just need to load a single model, to have access to a SpaCy `vocab` instance to load the reviews from so-called [DocBin](https://spacy.io/api/docbin)s.


In [85]:
# For now we'll just load the English parser model
lang_nlp = {lang: load_language_nlp_model(lang) for lang in ['en']}

# Reading book metadata and reviews

The review data contains a book identifier, so we know which book is associated with each review. There is a separate book metadata file with more info on the books, like title, author, publication year and statistics on ratings and reviews.

In [86]:
book_meta_file = '../data/book_metadata.csv'
book_df = pd.read_csv(book_meta_file, sep='\t')
book_df['book_author'] = book_df.book_author.apply(literal_eval)
book_df['book_author'] = book_df.book_author.apply(lambda x: x[0])
book_df.head(2)

,source_file,source_url,book_id,book_title,book_description,book_author,book_author_url,genres,format,num_pages,publication_date,rating_avg,rating_count,review_count,canonical_url
0,../data/Book_language_pages/en/19288043-gone-g...,https://www.goodreads.com/en/book/show/1928804...,19288043,Gone Girl,An alternative cover edition for this ISBN can...,Gillian Flynn,['https://www.goodreads.com/author/show/2383.G...,"['Fiction', 'Mystery', 'Thriller', 'Book Club'...",Paperback,415.0,2012-05-24T00:00:00,4.14,3399892,167690,https://www.goodreads.com/book/show/19288043-g...
1,../data/Book_language_pages/en/41865.Twilight....,https://www.goodreads.com/en/book/show/41865.T...,41865,Twilight,About three things I was absolutely positive. ...,Stephenie Meyer,['https://www.goodreads.com/author/show/941441...,"['Fantasy', 'Young Adult', 'Romance', 'Fiction...",Paperback,498.0,2005-10-05T00:00:00,3.67,7211130,146232,https://www.goodreads.com/book/show/41865.Twil...


Next, we load the reviews for the language specified above.

In [87]:
def read_reviews(review_file):
    with gzip.open(review_file, 'rt') as fh:
        return [json.loads(line) for line in fh]

review_dir = '../data/lang_reviews/'
review_files = glob.glob(os.path.join(review_dir, '*'))
review_file_map = {rf.split('lang_')[-1][:2]: rf for rf in review_files}
reviews = {}
for lang in lang_codes:
    reviews[lang] = read_reviews(review_file_map[lang])
    print(f"{len(reviews[lang])} reviews for language {lang}")

review_df = pd.DataFrame([review for lang in reviews for review in reviews[lang]])
review_df = pd.merge(review_df, book_df[['book_id', 'book_title', 'book_author']], on='book_id')

3495 reviews for language da
5825 reviews for language de
6270 reviews for language en
6254 reviews for language es
5481 reviews for language fa
5982 reviews for language fr
6128 reviews for language it
358 reviews for language ja
232 reviews for language ko
5861 reviews for language nl
5716 reviews for language pl
6131 reviews for language pt
764 reviews for language sl
5140 reviews for language sv
4277 reviews for language uk


A quick peek at the review data to know what it looks like:

In [88]:
review_df.head(2)

,review_text,user_id,review_id,review_date,shelf_status,user_shelves,rating,book_id,source_url,review_lang,book_title,book_author
0,"Meget fin, men har også følgende ting som irri...",a731b23ab9c845ad76c48c9ae0c37201af81a7294dd8f3...,8eee415ac1699108c550ba2b5804a44141c5b542dab4d9...,2025-01-26T00:00:00,Read,[],NaN,2175,https://goodreads.com/da/book/show/2175.Madame...,da,Madame Bovary,Gustave Flaubert
1,I jagten på at lykkes med livet kæmper vi med ...,9aeb2cd6b7ccc733d6d0c373c1c8e1d815a5ab40df3e07...,d2e728a22e350c07c7030f0ab53b9cf3ec646f6e25cfb3...,2025-04-11T00:00:00,,"[100-classics-penguin, 1001-books-boxall, 488-...",4.0,2175,https://goodreads.com/da/book/show/2175.Madame...,da,Madame Bovary,Gustave Flaubert


Next, we check per book how many reviews there are in each language. For many languages, there are 30 reviews per book. This is because the first page of reviews of a book contains at most 30 reviews. We have not crawled reviews beyond the first page, so 30 is the maximum number of reviews per book/language combination in our data set, but for many books there are many more reviews.

In [89]:
review_df.groupby(['book_id', 'book_title', 'book_author']).review_lang.value_counts().unstack().fillna(0.0)

,,review_lang,da,de,en,es,fa,fr,it,ja,ko,nl,pl,pt,sl,sv,uk
book_id,book_title,book_author,,,,,,,,,,,,,,,
11,The Hitchhiker’s Guide to the Galaxy,Douglas Adams,26.0,30.0,30.0,30.0,30.0,30.0,30.0,4.0,0.0,30.0,30.0,30.0,8.0,30.0,30.0
93,Heidi,Johanna Spyri,3.0,30.0,30.0,30.0,20.0,20.0,30.0,0.0,0.0,9.0,28.0,30.0,1.0,4.0,2.0
320,One Hundred Years of Solitude,Gabriel García Márquez,30.0,30.0,30.0,30.0,30.0,30.0,30.0,2.0,1.0,30.0,30.0,30.0,5.0,30.0,30.0
343,Perfume: The Story of a Murderer,Patrick Süskind,12.0,30.0,30.0,30.0,30.0,30.0,30.0,1.0,2.0,30.0,30.0,30.0,5.0,30.0,25.0
656,War and Peace,Leo Tolstoy,19.0,30.0,30.0,30.0,30.0,30.0,30.0,1.0,0.0,30.0,30.0,30.0,3.0,25.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61439040,1984,George Orwell,30.0,30.0,30.0,30.0,30.0,30.0,30.0,1.0,7.0,30.0,30.0,30.0,10.0,30.0,30.0
77265004,The Iliad,Homer,20.0,30.0,30.0,30.0,30.0,30.0,30.0,1.0,0.0,30.0,30.0,30.0,2.0,30.0,14.0
127441416,The Diary of a Young Girl,Anne Frank,30.0,30.0,30.0,30.0,30.0,30.0,30.0,2.0,0.0,30.0,30.0,30.0,4.0,30.0,30.0


In [90]:
# show an example review
print(json.dumps(reviews['it'][0], indent=4))

{
    "review_text": "C'EST MOI   Meravigliosa come sempre, semplicemente perfetta, Isabelle Huppert nell\u2019adattamento del 1991 firmato da Claude Chabrol. Letto un paio di volte e sempre amato. Uno dei massimi capolavori della letteratura, secondo me. Flaubert \u00e8 uno dei sommi: me lo immagino di notte, solo nella sua casa di Rouen, che sono ovviamente stato a visitare, al lume di candela, che 'recita' le parole scritte, ancora e ancora, urlandole, cancellando, limando, riscrivendo, fino a trovare la formula giusta, quella perfetta. Le mot juste.  Perch\u00e9, lui \u00e8 con la perfezione che si misurava.  E alla perfezione si \u00e8 avvicinato, e, secondo me, la perfezione ha raggiunto. Realistico, il romanzo certamente lo \u00e8: non contiene nulla che non sia esistito nella vita reale (e facilissimo da riscontrare attraverso sopralluoghi e testimonianze); e anche se sbuffa ogni tanto \"nulla in questa storia \u00e8 tratto dalla vita, \u00e8 totalmente inventata\", non c'\u00e

## Loading parsed reviews

As mentioned, the pre-parsed reviews are available in the `spacy_doc_bins` sub-directory. We load them as well, so we have access to the individual word tokens in each review, with per token the Part-Of-Speech (POS) tag and word lemma.

In [91]:


parsed_dir = '../data/spacy_doc_bins/'
parsed_reviews = {}
for lang in lang_codes:
    parsed_file = os.path.join(parsed_dir, f"parsed_reviews-{lang}.doc_bin")
    parsed_reviews[lang] = read_from_doc_bins(lang, parsed_dir, lang_nlp['en'].vocab)
    print(lang, len(parsed_reviews[lang]))

da 3495
de 5825
en 6270
es 6254
fa 5481
fr 5982
it 6128
ja 358
ko 232
nl 5861
pl 5716
pt 6131
sl 764
sv 5140
uk 4277


We also load stopwords lists, because our domain-specific vocabularies should contain domain-specific terms, not words common across all text genres in a language.

In [92]:
for lang in sorted(lang_codes):
    print(lang, len(stopwordsiso.stopwords(lang)))

da 170
de 620
en 1298
es 732
fa 799
fr 691
it 632
ja 134
ko 679
nl 413
pl 329
pt 560
sl 446
sv 418
uk 73


The next step is to build list of all non-stopwords used in reviews and keep track of the number of reviews in which they occur, so we can distinguish between words used in many reviews and words used in few reviews. The latter are probably to specific to a book or rare.

In [93]:
def filter_content_words(doc, stopwords):
    tokens = [token for token in doc if token.text not in stopwords and token.lemma_ not in stopwords]
    tokens = [token for token in tokens if token.pos_ != 'PUNCT' and token.lemma_ != ' ']
    return [token for token in tokens if len(token.text) > 2 and len(token.lemma_) > 2]

doc_freq = defaultdict(Counter)
for lang in parsed_reviews:
    stopwords = stopwordsiso.stopwords(lang)
    for doc in parsed_reviews[lang]:
        lemmas = [token.lemma_ for token in filter_content_words(doc, stopwords)]
        doc_freq[lang].update(set(lemmas))
        

If we remove stopwords, what are the most common words (words occurring in most documents)?

In English:

In [94]:
doc_freq['en'].most_common(10)

[('book', 4658),
 ('read', 4318),
 ('time', 3414),
 ('story', 3184),
 ('life', 2819),
 ('love', 2675),
 ('character', 2625),
 ('write', 2385),
 ('people', 2258),
 ('feel', 1893)]

In Italian:

In [95]:
doc_freq['it'].most_common(10)

[('libro', 3197),
 ('storia', 2128),
 ('leggere', 2108),
 ('di il', 2072),
 ('romanzo', 1983),
 ('potere', 1732),
 ('personaggio', 1709),
 ('a il', 1612),
 ('lettura', 1523),
 ('venire', 1411)]

In Ukranian:

In [96]:
doc_freq['uk'].most_common(10)

[('книга', 2079),
 ('той', 1425),
 ('історія', 1400),
 ('дуже', 1360),
 ('читати', 1349),
 ('свій', 1341),
 ('могти', 1153),
 ('життя', 1101),
 ('себе', 1065),
 ('людина', 984)]

The most common words in each language (where common is the number of reviews in which they occur) is clearly related to the domain of books and reading. 

### Build a vocabulary of common terms

To build an initial domain-specific vocabulary per language, we use a simple threshold: A term should occur in at least 1% of all reviews in a given language to be considered a domain term. Since we have a few thousand reviews for most languages, this corresponds to a threshold of a few dozen reviews. That ensure that terms are not specific to a single book, author or book series.

The chosen threshold of 1% is arbitrary. Depending on the total number of reviews in a language and the diversity of the books that they are associated with, you can try different thresholds. A good rule of thum is that a threshold should be substantially higher than the maximum number of reviews for a single book (preferably also higher than the number of reviews for a single book series or author), otherwise you may get plot-specific words or the names of characters or authors that are mentioned in many reviews of the same book, series or author.

In [97]:
min_df = {lang: len(parsed_reviews[lang]) * 0.01 for lang in parsed_reviews}
vocab = {}
for lang in doc_freq:
    vocab[lang] = set([term for term in doc_freq[lang] if doc_freq[lang][term] >= min_df[lang]])
    print(f"lang: {lang}  min_df: {min_df[lang]: >5.2f}  full vocab: {len(doc_freq[lang]): >7}"
          f"  common vocab: {len(vocab[lang]): >6}")

lang: da  min_df: 34.95  full vocab:   21921  common vocab:    513
lang: de  min_df: 58.25  full vocab:   47879  common vocab:    926
lang: en  min_df: 62.70  full vocab:   59070  common vocab:   2226
lang: es  min_df: 62.54  full vocab:   43067  common vocab:   1274
lang: fa  min_df: 54.81  full vocab:   57262  common vocab:   1290
lang: fr  min_df: 59.82  full vocab:   25074  common vocab:    724
lang: it  min_df: 61.28  full vocab:   48840  common vocab:   1319
lang: ja  min_df:  3.58  full vocab:    1880  common vocab:    160
lang: ko  min_df:  2.32  full vocab:    9573  common vocab:    560
lang: nl  min_df: 58.61  full vocab:   35978  common vocab:    557
lang: pl  min_df: 57.16  full vocab:   33724  common vocab:    584
lang: pt  min_df: 61.31  full vocab:   43304  common vocab:   1074
lang: sl  min_df:  7.64  full vocab:   10933  common vocab:    617
lang: sv  min_df: 51.40  full vocab:   26982  common vocab:    418
lang: uk  min_df: 42.77  full vocab:   36285  common vocab:   

The SpaCy docs with the parsed version of the review contain custom metadata including the identifier of the review. In the next steps we want to sort the reviews per book, so we add the book identifier to each SpaCy document for easy reference.

In [98]:
lang_book_docs = defaultdict(lambda: defaultdict(list))

for lang in parsed_reviews:
    for review, doc in zip(reviews[lang], parsed_reviews[lang]):
        doc.user_data['book_id'] = review['book_id']
        lang_book_docs[lang][doc.user_data['book_id']].append(doc)


### Language/Term vectors

Now that we have a domain-specific vocabulary per language, we can build vectors per language and term, where each element in the vector represents the number of reviews for a given book that contains the given term in the given language. That is, for each term in a language, the vector has 209 frequencies, one for each of the 209 books, representing the number of reviews containing that term.

First we make a list of all the book identifiers:

In [99]:
book_ids = sorted(set([review['book_id'] for lang in reviews for review in reviews[lang]]))

Next, we build a dictionary with language and term as key, and a list of 209 frequencies as value:

In [100]:
lang_term_freq = defaultdict(lambda: defaultdict(list))
for lang in parsed_reviews:
    if lang == 'zh':
        continue
    for book_id in book_ids:
        review_freq = Counter()
        for doc in lang_book_docs[lang][book_id]:
            common_tokens = set([token.lemma_ for token in doc if token.lemma_ in vocab[lang]])
        
            review_freq.update(common_tokens)
        for term in vocab[lang]:
            lang_term_freq[lang][term].append(review_freq[term])


Absolute frequencies are not directly comparable, because for some books there are more reviews than for others. Therefore, we want to turn the absolute frequencies into relative frequencies.

For that, we need to know how many reviews there are in total in each language for each book identifier:

In [101]:
lang_book_num_reviews = (review_df
    .groupby(['review_lang', 'book_id'])
    .book_id
    .value_counts()
    .unstack()
    .fillna(0.0))


Check how many reviews we have for each book in a given language, e.g. Italian:

In [102]:
lang_book_num_reviews.loc['it'].sort_values()

book_id
2767052       0.0
52516332      9.0
762390        9.0
17802724     14.0
6193         15.0
             ... 
17245        30.0
17690        30.0
18386        30.0
14942        30.0
239775146    30.0
Name: it, Length: 209, dtype: float64

Now we compute the relative frequency (fraction) vectors:

In [103]:
lang_term_frac = defaultdict(lambda: defaultdict(list))
for lang in lang_term_freq:
    for term in lang_term_freq[lang]:
        freq_num_reviews = zip(lang_term_freq[lang][term], lang_book_num_reviews.loc[lang])
        lang_term_frac[lang][term] = [freq / num_reviews if num_reviews > 0 else 0 for freq, num_reviews in freq_num_reviews]

Next, we build a matrix of the vectors, where the index of a row in the matrix corresponds to a language/term combination, and a column corresponds to a book identifier.

We make mappings from row indexes to term/language pairs and vice versa:

In [104]:
docid2term = {}
term2docid = {}

for lang in lang_term_frac:
    for term in lang_term_frac[lang]:
        docid = len(docid2term)
        docid2term[docid] = (lang, term)
        term2docid[(lang, term)] = docid

term_vecs = np.array([lang_term_frac[lang][term] for lang in lang_term_frac for term in lang_term_frac[lang]])
term_vecs.shape

(13387, 209)

There are 11,729 term/language pairs and 209 books.

To measure how similar two rows (two word/language pairs) are, we can use cosine similarity. **Note that with the multilingual term/language - book matrix, we can easily compute similarity of terms from different languages.

In [105]:
from sklearn.metrics.pairwise import cosine_similarity

# compute cosine similarity of the term/language vectors
term_cosim = cosine_similarity(term_vecs)

# make a dataframe with the term/language pairs as index and column labels
term_cosim = pd.DataFrame(term_cosim, columns=term2docid.keys(), index=term2docid.keys())


Now we can select a column (a term/language pair) and see which rows (term/language pairs) are most similar:

In [106]:
term_cosim[('en', 'character')].sort_values(ascending=False).head(20)

en  character      1.000000
es  personaje      0.948104
pt  personagem     0.942460
fr  personnage     0.940962
it  personaggio    0.928644
es  historia       0.920066
it  romanzo        0.915456
en  story          0.913704
    feel           0.912930
it  a il           0.910778
en  read           0.909190
it  da il          0.908457
en  love           0.908316
    time           0.908042
fa  داستان         0.907022
it  di il          0.904696
pt  livro          0.902955
    história       0.902281
fr  roman          0.901980
en  reader         0.899937
Name: (en, character), dtype: float64

The most similar terms to 'character' as equivalents in other languages, e.g. 'personage' in Spanish, 'personnage' in French and 'personaggio' in Italian. We don't see equivalents in most other languages, which might mean that in other languages there are no equivalents in the common vocabulary, or that the equivalents are used in different ways or at least in different contexts. 

We can also restrict the similarity to terms in a specific language:

In [107]:
term_cosim.loc['it'][('en', 'character')].sort_values(ascending=False).head(20)

personaggio    0.928644
romanzo        0.915456
a il           0.910778
da il          0.908457
di il          0.904696
in il          0.898537
storia         0.892953
potere         0.889716
riuscire       0.888808
libro          0.875448
venire         0.874147
su il          0.873745
leggere        0.872140
bello          0.866218
dovere         0.865983
piacere        0.864737
lettura        0.862533
trovare        0.862480
pagina         0.861608
andare         0.859184
Name: (en, character), dtype: float64

### Book similarity

We can also compute the similarity of books (in terms of how they are discussed in reviews using the domain-specific vocabulary). The 209 columns in the matrix represent book vectors, so if we transpose the matrix and compute cosine similarities for the 209 books based on the 11,729 term/language pairs.

First, we make more interpretable book labels for each book identifier, based on the title and author:

In [108]:
book_df['book_label'] = book_df.apply(lambda row: f"{row['book_author']}--{row['book_title']}", axis=1)
book_labels = book_df.sort_values('book_id').book_label

Next, we compute the similarities and make a dataframe with the book labels as index and column names:

In [109]:
doc_sim = cosine_similarity(term_vecs.T)
doc_sim = pd.DataFrame(doc_sim, columns=book_labels, index=book_labels)
doc_sim

book_label,Douglas Adams--The Hitchhiker’s Guide to the Galaxy,Johanna Spyri--Heidi,Gabriel García Márquez--One Hundred Years of Solitude,Patrick Süskind--Perfume: The Story of a Murderer,Leo Tolstoy--War and Peace,Arthur Golden--Memoirs of a Geisha,Dan Brown--Angels & Demons,Carlos Ruiz Zafón--The Shadow of the Wind,"John Gray--Men Are from Mars, Women Are from Venus",Homer--The Odyssey,...,Elizabeth Acevedo--Clap When You Land,Alice Walker--The Color Purple,Kristin Hannah--The Four Winds,Dan Brown--The Da Vinci Code,William Shakespeare--Romeo and Juliet,George Orwell--1984,Homer--The Iliad,Anne Frank--The Diary of a Young Girl,Jane Austen--Pride and Prejudice,Haruki Murakami--Kafka on the Shore
book_label,,,,,,,,,,,,,,,,,,,,,
Douglas Adams--The Hitchhiker’s Guide to the Galaxy,1.000000,0.437829,0.601506,0.562655,0.512045,0.607958,0.560127,0.595013,0.465449,0.576032,...,0.405234,0.551218,0.548871,0.612145,0.517875,0.593092,0.549699,0.560757,0.602864,0.606833
Johanna Spyri--Heidi,0.437829,1.000000,0.450984,0.424815,0.397115,0.495010,0.388086,0.435967,0.332437,0.457917,...,0.383017,0.468164,0.484684,0.429393,0.426209,0.440352,0.398108,0.455413,0.479182,0.425892
Gabriel García Márquez--One Hundred Years of Solitude,0.601506,0.450984,1.000000,0.559676,0.555989,0.629745,0.512285,0.622551,0.433032,0.615596,...,0.437771,0.577737,0.614229,0.572406,0.548149,0.608254,0.583873,0.589073,0.606318,0.634727
Patrick Süskind--Perfume: The Story of a Murderer,0.562655,0.424815,0.559676,1.000000,0.486820,0.587947,0.493926,0.554667,0.428738,0.534625,...,0.408016,0.555114,0.535818,0.563841,0.501141,0.565705,0.520332,0.519562,0.566424,0.553577
Leo Tolstoy--War and Peace,0.512045,0.397115,0.555989,0.486820,1.000000,0.542457,0.445479,0.526926,0.384188,0.553769,...,0.373378,0.495898,0.522876,0.501398,0.486810,0.550912,0.543661,0.531759,0.535411,0.534450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
George Orwell--1984,0.593092,0.440352,0.608254,0.565705,0.550912,0.603268,0.514492,0.579424,0.459637,0.597895,...,0.410103,0.554995,0.571974,0.565081,0.525442,1.000000,0.573138,0.599830,0.606841,0.593703
Homer--The Iliad,0.549699,0.398108,0.583873,0.520332,0.543661,0.550972,0.455519,0.537471,0.408575,0.676853,...,0.380023,0.526968,0.522572,0.525200,0.544329,0.573138,1.000000,0.552754,0.551606,0.540064
Anne Frank--The Diary of a Young Girl,0.560757,0.455413,0.589073,0.519562,0.531759,0.607424,0.479290,0.548995,0.452341,0.566657,...,0.441320,0.560778,0.593894,0.542178,0.525969,0.599830,0.552754,1.000000,0.570428,0.556246


Now, we can pick any book label and query the dataframe to find books that are similar in terms of the domain-specific vocabulary:

In [111]:
book_label = "John Green--The Fault in Our Stars"

doc_sim[book_label].sort_values(ascending=False).head(20)

book_label
John Green--The Fault in Our Stars                                                      1.000000
Haruki Murakami--Norwegian Wood                                                         0.691071
Jojo Moyes--Me Before You                                                               0.681216
Stephenie Meyer--Twilight                                                               0.675204
Gabriel García Márquez--Love in the Time of Cholera                                     0.659274
Milan Kundera--The Unbearable Lightness of Being                                        0.656757
Ray Bradbury--Fahrenheit 451                                                            0.653570
Khaled Hosseini--The Kite Runner                                                        0.653081
Terry Pratchett--Good Omens: The Nice and Accurate Prophecies of Agnes Nutter, Witch    0.651623
J.K. Rowling--Harry Potter and the Cursed Child: Parts One and Two                      0.650939
Paulo Coelho--The A

In [112]:
book_label = "Anne Frank--The Diary of a Young Girl"

doc_sim[book_label].sort_values(ascending=False).head(20)

book_label
Anne Frank--The Diary of a Young Girl                    1.000000
John Green--The Fault in Our Stars                       0.638703
Ray Bradbury--Fahrenheit 451                             0.615735
Khaled Hosseini--A Thousand Splendid Suns                0.610623
Paulo Coelho--The Alchemist                              0.608170
Haruki Murakami--Norwegian Wood                          0.608125
Arthur Golden--Memoirs of a Geisha                       0.607424
Margaret Atwood--The Handmaid's Tale                     0.607268
Vladimir Nabokov--Lolita                                 0.604027
George Orwell--1984                                      0.599830
Kristin Hannah--The Four Winds                           0.593894
Albert Camus--The Plague                                 0.593798
Leo Tolstoy--Anna Karenina                               0.590644
Khaled Hosseini--The Kite Runner                         0.590228
Hermann Hesse--Steppenwolf                               0.589896

## Term-Document matrix

Finally, we can turn the term-document matrix into a dataframe as well, which allows us to:

- use a vocabulary term to find which books are most commonly described by that term (e.g. for which books do reviewers often mentioned characters, plot or writing style)
- use a book label to find which vocabulary terms are most commonly used to describe it (e.g. what terms are most typically used to describes the diary of Anna Frank).

In [113]:
term_doc_frac = pd.DataFrame(term_vecs, columns=book_labels, index=term2docid.keys())

term_doc_frac.loc[('en', 'character')].sort_values(ascending=False).head(20)

book_label
Liu Cixin--The Three-Body Problem                 0.833333
Leo Tolstoy--Anna Karenina                        0.833333
Chimamanda Ngozi Adichie--Half of a Yellow Sun    0.800000
Chimamanda Ngozi Adichie--Americanah              0.800000
Stephen        King--Finders Keepers              0.766667
Victor Hugo--Les Misérables                       0.766667
Thomas Mann--The Magic Mountain                   0.766667
Jane Austen--Mansfield Park                       0.733333
Fyodor Dostoevsky--The Brothers Karamazov         0.733333
Stieg Larsson--The Girl With the Dragon Tattoo    0.733333
Stieg Larsson--The Girl Who Played with Fire      0.700000
Gillian Flynn--Dark Places                        0.700000
Stephen        King--Mr. Mercedes                 0.700000
Isabel Allende--The House of the Spirits          0.700000
Anthony Doerr--All the Light We Cannot See        0.700000
Leo Tolstoy--War and Peace                        0.700000
Agatha Christie--Death on the Nile           

In [114]:
book_label = "Anne Frank--The Diary of a Young Girl"

term_doc_frac[book_label].loc[['en']].sort_values(ascending=False).head(20)

en  book      0.766667
    Anne      0.700000
    read      0.700000
    diary     0.700000
    time      0.666667
    people    0.600000
    life      0.600000
    write     0.533333
    girl      0.500000
    live      0.466667
    family    0.433333
    word      0.433333
    day       0.433333
    war       0.400000
    review    0.400000
    happen    0.366667
    story     0.366667
    feel      0.366667
    heart     0.333333
    love      0.333333
Name: Anne Frank--The Diary of a Young Girl, dtype: float64

In [115]:
term_doc_frac[book_label].loc[['it']].sort_values(ascending=False).head(20)

it  di il        0.833333
    a il         0.700000
    diario       0.666667
    in il        0.600000
    leggere      0.533333
    da il        0.500000
    dovere       0.466667
    potere       0.466667
    libro        0.466667
    ragazzino    0.433333
    volere       0.366667
    su il        0.366667
    sapere       0.366667
    pagina       0.366667
    guerra       0.366667
    piccolo      0.333333
    età          0.333333
    vivere       0.333333
    persona      0.333333
    pensiero     0.300000
Name: Anne Frank--The Diary of a Young Girl, dtype: float64